# LLM

> A module to make direct queries to OpenAI, and eventually other LLMs.


This core notebook is intended to do reading and querying from OpenAI api, once you've obtained your domain data using the code from `police_risk_open_ai.core.crawl`

In [ ]:
#| default_exp llm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
from dotenv import load_dotenv
import pandas as pd
import openai
import tiktoken
from tqdm import tqdm
import numpy as np
from openai.embeddings_utils import distances_from_embeddings


To run OpenAI, you'll need:


- the openai python library
- an API key, saved as an environment variable

In [ ]:
#| export

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
#| export
def query_llm(prompt,model="text-davinci-003"):
    "Given a prompt, will query OpenAI and return the output"
    
    response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0, max_tokens=7)

    text = response['choices'][0]['text']
    return text


In [ ]:
query_llm("What is a bear?")

'\n\nA bear is a large'

That all works! I can now run `nbdev_prepare` to prepare my code to commit.

## Ingesting Data
LLMs don't know anything about how to conduct risk assessments, so we're going to make ours "read" the entirety of [Authorised Professional Practice](https://www.college.police.uk/app/using-app). For now I'm mostly copying this [OpenAI tutorial](https://platform.openai.com/docs/tutorials/web-qa-embeddings).  That's covered in the second notebook, but I've scrapped and clean the College of Policing website.  You can see that in the other module.

In [ ]:
#|eval: false

df = pd.read_parquet('processed/embeddings.parquet')

Before we can begin analysis, we need to split our text into `tokens`, recognisable chunks of text-data our model will recognise.

That's it!  We've converted our entire library to arrays, and you can see each document converted into our series of datapoints.  I'll export that code into a function.

## Questions!
With our embedding process completed, let's convert them to Numpy arrays.

We're now back and close to following the tutorial. The approach is actually quite clever: we use the embeddings to find documents most likely to related, and then combined them into a prompt for completion.

In [ ]:
#| export
def create_context(
    question, df, max_len=1800, size="ada"
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')


    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

In [ ]:
#| export

def answer_question(
    df,
    model="text-davinci-003",
    question="Am I allowed to publish model outputs to Twitter, without a human review?",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=150,
    stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )
    # If debug, print the raw model response
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.Completion.create(
            prompt=f"Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:",
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

In [ ]:
#|eval: false

answer_question(df, question="What day is it?", debug=True)

Context:
.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

.

###

It’s also been raining heavily in the night and we have further calls about flooding in the road, so we ring Highways to inform them. I have a little smile to myself as I remember a call in the summer about cars stopping on the M11 because a mother duck and her ducklings were crossing the road. Lunchtime looms. I’m feeling hungry, but that disappears when I take a call from a 16-year-old male, who tells me that he can’t cope any more. He has cut himself with a knife but he doesn’t want to die. His sister has just had a baby. This goes on an emergency straight away, and officers are dispatched within three minutes. I have to talk to him about anything I can to distract him from his misery – luckily, I am good at small 

"I don't know."

In [ ]:
#|eval: false

answer_question(df, question="What are the most important factors to consider when searching for a missing person?", debug=True)

Context:
Very detailed information and a lifestyle profile will be needed in high-risk cases consider taking a full statement from the person reporting the missing person as well as any other key individuals (for example, the last person to see them) conduct initial searches of relevant premises, the extent and nature of the search should be recorded (see Search) consider seizing electronic devices, computers, and other documentation, (for example, diaries, financial records and notes) and obtain details of usernames and passwords obtain photos of the missing person; these should ideally be current likeness of the missing person and obtained in a digital format obtain details of the individual’s mobile phone and if they have it with them; if the missing person has a mobile phone arrange for a TextSafe© to be sent by the charity, Missing People obtain details of any vehicles that they may have access to and place markers on relevant vehicles on the PNC without delay consider obtaining a

'It is important to adopt an investigative approach to all reports, ensuring that assumptions are not made about the reasons for going missing. The importance and relevance of risk factors will depend on the circumstances of each case and require investigation to determine if there is a cause for concern. It is also important to record the name and contact details of the person who gave that information and when this happened. Missing people may be at risk of harm resulting from factors such as: an inability to cope with weather conditions, being the victim of violent crime, and risks relating to non-physical harm, for example, the people they are with, the places or circumstances they are in.'

Now, let's tweak to answer sergeant exam questions, and see how it does.

In [ ]:
#| export

def answer_sergeant_exam_question(
    df,
    question,
    model="text-davinci-003",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=150,
    stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """

    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )

    exam_prompt= f""" Answer the question below, in the format 'The answer is X because Z', where X is the letter of the correct answer (A,B,C or D) and X is the explanation in fewer than 3 sentences. 

    if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:"""

    # If debug, print the raw model response
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.Completion.create(
            prompt=exam_prompt,
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

Let's answer a question from here
https://www.how2become.com/blog/police-sergeants-inspectors-exam/


In [ ]:
#|eval: false

question = """ Officer Jennings is on his evening patrol. He is just about to finish for the day. As he walks down the street, he is approached by a man named Mark, who claims that he saw a man (named Steven) driving down a road not far from the location. Mark claims that he saw Steven drive into a cyclist, before driving off without stopping. Luckily, the cyclist was unharmed. The cyclist was named Kevin. Mark spoke to Kevin, and discovered that he is a 42 year old man, with a wife and two daughters.

Fifteen minutes later, Officer Jennings manages to stop the car being driven by Steven. He pulls him over to the side of the road, and orders him to step out of the car.

 

Referring s.6 (5) of the Road Traffic Act 1988, is Officer Jennings within his legal rights to order that Steven takes a preliminary breath test?

A – No. Officer Jennings has no right to tell Steven what he can and can’t do. He should never have stopped Steven in the first place.

B – No. In order for Officer Jennings to do this, an accident must have happened. The fact that Officer Jennings suspects an accident has taken place, does not meet this requirement.

C – Yes. However, the breath test must take place within or close to an area where the requirements for Steven to cooperate, can be imposed.

D – Yes. Officer Jennings can tell Steven to do whatever he wants, as he’s a police officer."""

answer_sergeant_exam_question(df,question)

'B - No. In order for Officer Jennings to do this, an accident must have happened. The fact that Officer Jennings suspects an accident has taken place, does not meet this requirement.'

In [ ]:
#| export

def conduct_risk_assessment(
    df,
    question,
    model="text-davinci-003",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=150,
    stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """

    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )

    missing_risk_prompt = f""" Using the information below on a missing person, decide on the appropriate risk grading for the person, from either
- High risk
- Medium risk
- Low risk
- no apparent risk

Return your answer in the format: 'Graded as X risk, because of the below risk factors:
- Y
- Z'

Where X is your risk grading (high, medium, low, or no apparent risk) and Y and Z are a few sentences explaining the most important risks you have identified.

if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:""",

    # If debug, print the raw model response
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.Completion.create(
            prompt=missing_risk_prompt,
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

In [ ]:
#|eval: false

risk_profile = """John Smith is a 31 year old man who has been missing for 5 hours. He went to work as normal this morning, and has not returned home, and his partner is concerned. There are no signs of foul play, no vulnerabilities. John is in good health, the weather is good, and there are no concerns for his welfare."""


conduct_risk_assessment(df, risk_profile, debug=True)

'Graded as Low risk, because of the below risk factors:\n- John is an adult, and there are no signs of foul play or vulnerabilities\n- He is in good health, and the weather is good\n- There are no concerns for his welfare'

In [ ]:
#|eval: false

jason_risk_profile = """ Jason is a 15 year old adult male, who has gone missing from his care home in Southwark. His carer has contacted the school, which has said he was not in today.
They that this is not the first time, and that Jason has been seen hanging out with older boys, who may be involved in crime and drugs."""


conduct_risk_assessment(df, jason_risk_profile, debug=True)

Context:
 First published 22 November 2016  Updated 15 March 2023   Latest changes  Written by College of Policing  Missing persons  30 mins read   Implications for the UK leaving the European Union are currently under review – please see APP on international investigation for latest available detail on specific areas, for example: Schengen Information System Europol INTERPOL Joint Investigation Teams This section provides additional information to aid the investigation based on the vulnerability of the individual and the circumstances in which they are missing. Missing children Safeguarding young and vulnerable people is a responsibility of the police service and partner agencies (see Children Act 2004). When the police are notified that a child is missing, there is a clear responsibility on them to prevent the child from coming to harm. Where appropriate, a strategy meeting may be held. For further information see: Voice of the child  Voice of the child practice briefing  Section 11 

'Graded as Medium risk, because of the below risk factors:\n- Jason is a 15 year old adult male, who has gone missing from his care home in Southwark\n- This is not the first time he has gone missing\n- He has been seen hanging out with older boys, who may be involved in crime and drugs'

The model works.  I'm happy with it.  So let's tie it into a function we can deploy as a web app.

In [ ]:
#| export

def machine_risk_assessment(
    question,
    df,
    model="text-davinci-003",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=150,
    stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """

    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )

    missing_risk_prompt = f""" Using the information below on a missing person, decide on the appropriate risk grading for the person, from either
- High risk
- Medium risk
- Low risk
- no apparent risk

Return your answer in the format: 'Graded as X risk, because of the below risk factors:\n - Y \n - Z \n'

Where X is your risk grading (high, medium, low, or no apparent risk) and Y and Z are a few sentences explaining the most important risks you have identified.

if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:""",

    # If debug, print the raw model response

    
    if debug:
        print("Question:\n" + question)
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.Completion.create(
            prompt=missing_risk_prompt,
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )

        answer = response["choices"][0]["text"].strip()
        
        return answer, context
    except Exception as e:
        print(e)
        return ""

In [ ]:
#|eval: false

about_yannik = """ Yannik is a 15 year old boy. He has recently been down, and was reported missing by his parents as he did not return home from school today.

His friends are worried he may be depressed, and when he apparently told one a few days ago 'if it doesn't get any better, I'm going to end it soon'
"""

yannik_answer, yannik_context = machine_risk_assessment(about_yannik, df)
yannik_answer

'Graded as High risk, because of the below risk factors: \n- Yannik is a 15 year old boy who has been reported missing by his parents\n- His friends are worried he may be depressed\n- He has expressed suicidal ideation to one of his friends'

It works!

Let's now add the ChatAPI rather than auto-completion, which should improve performance and cost

In [ ]:
#| export
copbot_chat_content = '''
You are CopBot, an assistant designed to help police officers risk assess missing persons.

Using the information provide on a missing person, you will decide on the appropriate risk grading for the person, from either
- No apparent risk (when there is no apparent risk of harm to either the subject or the public.)
- Low risk (when the risk of harm to the subject or the public is assessed as possible but minimal)
- Medium risk (when the risk of harm to the subject or the public is assessed as likely but not serious.)
- High risk (when the risk of serious harm to the subject or the public is assessed as very likely.)

Risk assessment should be guided by the College of Policing Risk principles.'''



In [ ]:
#| export
copbot_question_intro = ''' Here are some details of a missing person:

'''

In [ ]:
#| export
copbot_question_outro = '''

Based on the above, please provide a risk assessment for the missing person, guided by the College of Policing Risk principles, which is either:
- No apparent risk 
- Low risk
- Medium risk
- High risk

Return your answer in the format: 

'Graded as X risk, because of the below risk factors:\n - Y \n - Z \n \n Given these factors...'

Where X is your risk grading (high, medium, low, or no apparent risk) and Y and Z are a few sentences explaining the most important risks you have identified.

Always return your answer in this format, unless the question can't be answered based on the context, say \"I don't know\"'''



In [ ]:
#| export
def create_chat_assistant_content(
    question, df, max_len=2800, size="ada"
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')


    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

In [ ]:
#| export
def create_chat_assistant_question(question):

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')


    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

In [ ]:
#| export
def copbot_chat_risk_assessment(individual_circumstances, df, return_context=False, debug_mode=False):
    """Takes a user input string about the individual circumstances of a missing person, and returns a risk assessment"""

    individual_context = create_chat_assistant_content(individual_circumstances, df)

    question_and_context = copbot_question_intro + individual_circumstances + copbot_question_outro

    openai_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": copbot_chat_content},
            {"role": "user", "content": question_and_context},
            {"role": "assistant", "content": individual_context},
        ]
    )

    if debug_mode:
        print(openai_response)
        print('\n\n\n')

    if return_context:
        return openai_response['choices'][0]['message']['content'], individual_context
    
    else:
        return openai_response['choices'][0]['message']['content']

    

In [ ]:
#|eval: false

df = pd.read_parquet('processed/embeddings.parquet')


jason_risk_profile = """Jason is a 15 year old adult male, who has gone missing from his care home in Southwark. His carer has contacted the school, which has said he was not in today.
They that this is not the first time, and that Jason has been seen hanging out with older boys, who may be involved in crime and drugs."""

copbot_chat_risk_assessment(jason_risk_profile, df, debug_mode=True)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Graded as medium risk, because of the below risk factors:\n- The person is a minor and vulnerable\n- The person has been seen hanging out with older boys who may be involved in crime\n\nGiven these factors, there is a likelihood of harm to the subject as well as the public, especially if he is being influenced by the older boys he has been seen with. Therefore, appropriate immediate action is required to locate and ensure the safety of Jason. Additionally, it may be necessary to investigate the activities of the care home to establish how he was allowed to go missing repeatedly.",
        "role": "assistant"
      }
    }
  ],
  "created": 1680974755,
  "id": "chatcmpl-736bzcixDS24mIRSX3ncHR9ARrknH",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 116,
    "prompt_tokens": 2706,
    "total_tokens": 2822
  }
}






'Graded as medium risk, because of the below risk factors:\n- The person is a minor and vulnerable\n- The person has been seen hanging out with older boys who may be involved in crime\n\nGiven these factors, there is a likelihood of harm to the subject as well as the public, especially if he is being influenced by the older boys he has been seen with. Therefore, appropriate immediate action is required to locate and ensure the safety of Jason. Additionally, it may be necessary to investigate the activities of the care home to establish how he was allowed to go missing repeatedly.'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()